## Descritores de textura - Índices Taxonômicos

<b>Importando as bibliotecas</b>

In [1]:
import glob as g
from skimage.io import imread
from skimage.color import rgb2gray
import numpy as np

<b>Método para salvar o arquivo de características em libsvm</b>

In [2]:
def geraSVMfile(rotulo, lista_feat, name_file, path_out, modo):
    ''''''
    arquivo = open(path_out + name_file, modo)
    featureFile = str(rotulo) + " "
    arquivo.write(featureFile)
    for i in range(len(lista_feat)):
        linha = str(str(i + 1) + ":" + str(lista_feat[i]) + " ")
        print('linha ', str(i + 1), " = ", linha)
        arquivo.write(linha)
    arquivo.write('\n')
    arquivo.close()

<b>Extraindo as características</b>

In [3]:
def run_imagens(classe, lista, modo):

    somatorio = 0
    MNND = 0.0
    histograma = [0]*256
    especie_ = 0

    for i in range(len(lista)):
        
        name = lista[i].split('/')[-1].split('.')[0].split('g')[0]

        img = imread(lista[i])
        img = rgb2gray(img)*255
        img = np.uint32(img)
        
        min = img.min()
        sizeHistograma = 256

        for b in range(img.shape[0]):
            for c in range(img.shape[1]):
                value = img[b][c]
                if int(value) > 0:
                    histograma[value] += 1

        for d in range(0, sizeHistograma):
            if int(histograma[d]) > 0:
                especie_ += int(histograma[d])

        # diversidade taxonômica
        somatorioDelta = 0.0
        for x in range(0, sizeHistograma):
            somatorio_initial = 0.0
            for y in range(x + 1, sizeHistograma):
                val = histograma[x] * histograma[y] * ((y - x) + 1)
                somatorio_initial += val
            somatorioDelta += somatorio_initial
        deLta = somatorioDelta / (especie_ * (especie_ - 1) / 2)

        # distinção taxonômica
        somatorioDelta_ = 0.0
        somatorioDelta_q = 0.0
        for x in range(0, sizeHistograma):
            somatorio_initial_ = 0.0
            somatorio_initialq = 0.0
            for y in range(x + 1, sizeHistograma):
                val = histograma[x] * histograma[y] * ((y - x) + 1)
                valq = histograma[x] * histograma[y]
                somatorio_initial_ += val
                somatorio_initialq += valq
            somatorioDelta_ += somatorio_initial_
            somatorioDelta_q += somatorio_initialq
        delta_ = somatorioDelta_ / somatorioDelta_q

        # lista de características
        features = []
        features.append(deLta)
        features.append(delta_)

        # Salvando as características no arquivo libsvm
        print('imagem ' + str(i) + ' da classe ' + str(classe) + ' foi processada ')
        name = 'taxonomicos.libsvm'
        path_file = '/home/vitoria/Área de Trabalho/ICV/SINFO/resultados/'
        geraSVMfile(rotulo=classe, lista_feat=features, name_file=name, path_out=path_file, modo=modo)


<b>Obtendo as imagens da base</b>

In [4]:
def getImagens():
    #mudar os caminhos assim que eu conseguir a base.
    path_melanoma = '/home/vitoria/Área de Trabalho/ICV/SINFO/ph2/melanoma/'
    path_nmelanoma = '/home/vitoria/Área de Trabalho/ICV/SINFO/ph2/naomelanoma/'
    extensao = '*.bmp'

    lista_melanoma = g.glob(path_melanoma + extensao)
    lista_nmelanoma = g.glob(path_nmelanoma + extensao)

    run_imagens(classe=1, lista=lista_melanoma, modo='a')
    run_imagens(classe=0, lista=lista_nmelanoma, modo='a')

    #melanoma => classe 1
    #não melanoma => classe 0

In [5]:
if __name__ == '__main__':
    
    getImagens()

imagem 0 da classe 1 foi processada 
linha  1  =  1:33.48883633262873 
linha  2  =  2:33.84329329562838 
imagem 1 da classe 1 foi processada 
linha  1  =  1:19.521872194212484 
linha  2  =  2:40.43040114811073 
imagem 2 da classe 1 foi processada 
linha  1  =  1:14.359581295268251 
linha  2  =  2:33.61889211136817 
imagem 3 da classe 1 foi processada 
linha  1  =  1:9.68207745765739 
linha  2  =  2:37.949208824614324 
imagem 4 da classe 1 foi processada 
linha  1  =  1:8.868617880630726 
linha  2  =  2:48.22142877294503 
imagem 5 da classe 1 foi processada 
linha  1  =  1:5.910029448849589 
linha  2  =  2:48.04900139014076 
imagem 6 da classe 1 foi processada 
linha  1  =  1:3.8628068265008637 
linha  2  =  2:47.278896415459194 
imagem 7 da classe 1 foi processada 
linha  1  =  1:2.457802979851492 
linha  2  =  2:46.71315852821642 
imagem 8 da classe 1 foi processada 
linha  1  =  1:2.064579752962954 
linha  2  =  2:46.13394903696284 
imagem 9 da classe 1 foi processada 
linha  1  =  1